# ✨🦙 Build LLM applications with LlamaIndex and monitor the data with Argilla.

This integration allows the user to include the feedback loop that Argilla offers into the LlamaIndex ecosystem. It's based on a callback handler to be run within the LlamaIndex workflow. 

Don't hesitate to check out both [LlamaIndex](https://github.com/run-llama/llama_index) and [Argilla](https://github.com/argilla-io/argilla)

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:


**Deploy Argilla on Hugging Face Spaces**: If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).


**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](../../getting_started/quickstart.ipynb). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>

## Getting started

You first need to install argilla and argilla-llama-index as follows:

In [ ]:
%pip install argilla-llama-index

You will need to an Argilla Server running to monitor the LLM. You can either install the server locally or have it on HuggingFace Spaces. For a complete guide on how to install and initialize the server, you can refer to the [Quickstart Guide](https://docs.argilla.io/en/latest/getting_started/quickstart_installation.html).

## Usage

It requires just a simple step to log your data into Argilla within your LlamaIndex workflow. We just need to call the handler before starting production with your LLM.

We will use GPT3.5 from OpenAI as our LLM. For this, you will need a valid API key from OpenAI. You can have more info and get one via [this link](https://openai.com/blog/openai-api).

After you get your API key, the easiest way to import it is through an environment variable, or via *getpass()*.

In [1]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

Let's now write all the necessary imports and initialize the Argilla client

In [2]:
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader, set_global_handler
from llama_index.llms.openai import OpenAI

import argilla as rg

rg.init(
    api_url="http://localhost:6900",
    api_key="owner.apikey",
    workspace="admin"
)

Now, we will set up an Argilla global handler for Llama Index. By doing so, we ensure that the predictions that we obtain using Llama Index is automatically uploaded to the Argilla client we initialized before Within the handler, we need to provide the dataset name that we will use. If the dataset does not exist, it will be created with the given name. You can also set the API KEY, API URL, and the Workspace name. You can learn more about the variables that controls Argilla initialization [here](https://docs.argilla.io/en/latest/getting_started/installation/configurations/workspace_management.html).

In [4]:
set_global_handler("argilla", dataset_name="query_model")

[04/09/24 16:26:24] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=154751;file:///Users/ignacio/Documents/recognai/argilla-llama-index/.venv/lib/python3.10/site-packages/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=190584;file:///Users/ignacio/Documents/recognai/argilla-llama-index/.venv/lib/python3.10/site-packages/argilla/client/feedback/dataset/local/mixins.py#271\271]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=994278;file:///Users/ignacio/Documents/recognai/argilla-llama-index/.venv/lib/python3.10/site-packages/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=162794;file:///Users/ignacio/Documents/recognai/argilla-llama-index/.venv/lib/python3.10/site-packages/argilla/client/feedback/dataset/local/mixins.py#272\272]8;;\
                                id=f3ca83bf-14c0-49a8-9d30-7bdd7fe88cf0                                            
                                name=query_model                                                                   
                                workspace=Workspace(id=5b42d217-c190-4916-835b-924c9eb766ef,                       
                             name=admin, inserted_at=2024-04-09 08:59:41.374316,                                   
                             updated_at=2024-04-09 08:59:41.374316)                                                
                                url=http://localhost:6900/dataset/f3ca83bf-14c0-49a8-9d30-7bdd7fe88cf              
                             0/annotation-mode                                                                     
                                fields=[RemoteTextField(id=UUID('e199de61-ec75-43b3-85b8-75ca82f95d5d              
                             '), client=None, name='prompt', title='Prompt', required=True,                        
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('fc7e8a06-7778-47b0-bbd3-69347586e951'),                      
                             client=None, name='response', title='Response', required=False,                       
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('c65d166a-f9e4-4270-853e-68d935deee64'),                      
                             client=None, name='time-details', title='Time Details', required=True,                
                             type='text', use_markdown=True)]                                                      
                                questions=[RemoteRatingQuestion(id=UUID('199e8908-2c37-4e86-bf6e-a9dd              
                             c8813f6a'), client=None, name='response-rating', title='Rating for the                
                             response', description='How would you rate the quality of the                         
                             response?', required=True, type='rating', values=[1, 2, 3, 4, 5, 6, 7]),              
                             RemoteTextQuestion(id=UUID('82111561-fe34-44ad-952e-77d532152ebc'),                   
                             client=None, name='response-feedback', title='Feedback for the                        
                             response', description='What feedback do you have for the response?',                 
                             required=False, type='text', use_markdown=False)]                                     
                                guidelines=You're asked to rate the quality of the response and                    
                             provide feedback.                                                                     
                                metadata_properties=[]                                                             
                                vectors_settings=[]                                                                
                             )                                                                                     

/Users/ignacio/Documents/recognai/argilla-llama-index/src/argilla_llama_index/llama_index_handler.py:135: UserWarning: No dataset with the name query_model was found in workspace admin. A new dataset with the name query_model has been created with the question fields `prompt` and `response`and the rating question `response-rating` with values 1-7 and text question named `response-feedback`.
  warnings.warn(


Let's now create the llm instance, using GPT-3.5 from OpenAI.

In [5]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.8, openai_api_key=openai_api_key)

With the code snippet below, you can create a basic workflow with LlamaIndex. You will also need a txt file as the data source within a folder named "data". We have an example `.txt` file ready for you inside that folder, obtained from the [Llama Index documentation](https://docs.llamaindex.ai/en/stable/getting_started/starter_example.html).

In [6]:
service_context = ServiceContext.from_defaults(llm=llm)
docs = SimpleDirectoryReader("../../data").load_data()
index = VectorStoreIndex.from_documents(docs, service_context=service_context)
query_engine = index.as_query_engine()

/var/folders/s0/4h4_2s7n3wxbm06wx8y43mm80000gn/T/ipykernel_24151/2413633764.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm)


Now, let's run the `query_engine` to have a response from the model.

In [7]:
response = query_engine.query("What did the author do growing up?")
response

/Users/ignacio/Documents/recognai/argilla-llama-index/.venv/lib/python3.10/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Response(response='The author, growing up, worked on writing short stories and programming. Initially, he wrote short stories with minimal plots but strong character emotions. He then delved into programming on an IBM 1401 using an early version of Fortran, creating programs that had limited functionality due to the constraints of the system. Later, with the introduction of microcomputers, he transitioned to programming on more user-friendly platforms like the TRS-80, where he developed simple games and a word processor.', source_nodes=[NodeWithScore(node=TextNode(id_='9e4d418c-649e-4317-9cc5-777fc659528a', embedding=None, metadata={'file_path': '/Users/ignacio/Documents/recognai/argilla-llama-index/docs/examples/../data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75041, 'creation_date': '2024-03-25', 'last_modified_date': '2024-02-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_

The prompt given and the response obtained will be logged in to Argilla server. You can check the data on Argilla's UI:

![Argilla Dataset](../../_static/tutorials/llama_index/argilla-ui-dataset.png)